In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Train Test Split

In [43]:
raw = pd.read_csv("datasets/all_features.csv")
raw = raw.drop(columns = ["SOFR","SOFRVOL","EFFRVOL","OBFR","OBFRVOL","EFFRVOL_DIFF"
                          ,"OBFR_DIFF","OBFRVOL_DIFF","SP500_UNNORM", "SOFR_DIFF", "SOFRVOL_DIFF"])

In [46]:
raw.dropna()

,date,AAA10Y,ASEANTOT,BAA10Y,BUSAPPWNSAUS,BUSAPPWNSAUSYY,CBUSAPPWNSAUS,CBUSAPPWNSAUSYY,CUUR0000SA0R,DEXCHUS,...,PCUOMFGOMFG_DIFF,PCUOMINOMIN_DIFF,SFTPAGRM158SFRBSF_DIFF,T10YIE_DIFF,TEDRATE_DIFF,TLAACBW027NBOG_DIFF,TLBACBW027NBOG_DIFF,TSIFRGHT_DIFF,UNRATE_DIFF,WLEMUINDXD_DIFF
3222,2011-03-16,0.528072,1.607233,0.123031,0.245383,-0.088867,0.259760,-0.155651,-0.388576,-0.759879,...,0.172509,0.183195,0.023427,-0.072674,0.000000,-0.019734,-0.023882,0.138321,0.000000,-0.891974
3223,2011-03-17,0.528072,1.607233,0.136493,0.245383,-0.088867,0.259760,-0.155651,-0.388576,-0.756144,...,0.172509,0.183195,0.023427,0.145347,0.050066,-0.019734,-0.023882,0.138321,0.000000,-0.321332
3224,2011-03-18,0.504994,1.607233,0.096106,0.245383,-0.088867,0.259760,-0.155651,-0.388576,-0.762867,...,0.172509,0.183195,0.023427,0.000000,0.025033,-0.019734,-0.023882,0.138321,0.000000,-0.070541
3226,2011-03-21,0.412685,1.607233,0.028795,0.177960,-0.114123,0.428271,-0.125545,-0.388576,-0.769589,...,0.172509,0.183195,0.023427,0.024225,-0.075099,-0.019734,-0.023882,0.138321,0.000000,0.181138
3227,2011-03-22,0.389607,1.607233,0.028795,0.177960,-0.114123,0.428271,-0.125545,-0.388576,-0.785151,...,0.172509,0.183195,0.023427,-0.121123,0.000000,-0.019734,-0.023882,0.138321,0.000000,0.244378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6332,2021-02-05,-0.302713,-0.579975,-0.590471,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,0.096898,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386
6333,2021-02-08,-0.348868,-0.579975,-0.644321,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,0.024225,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386
6334,2021-02-09,-0.348868,-0.579975,-0.630858,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,0.000000,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386
6335,2021-02-10,-0.325791,-0.579975,-0.617396,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,-0.024225,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386


In [47]:
train = raw[raw["date"] < "2018-00-00"]

,date,AAA10Y,ASEANTOT,BAA10Y,BUSAPPWNSAUS,BUSAPPWNSAUSYY,CBUSAPPWNSAUS,CBUSAPPWNSAUSYY,CUUR0000SA0R,DEXCHUS,...,PCUOMFGOMFG_DIFF,PCUOMINOMIN_DIFF,SFTPAGRM158SFRBSF_DIFF,T10YIE_DIFF,TEDRATE_DIFF,TLAACBW027NBOG_DIFF,TLBACBW027NBOG_DIFF,TSIFRGHT_DIFF,UNRATE_DIFF,WLEMUINDXD_DIFF
3196,2011-02-14,0.043447,1.280784,-0.119291,0.267132,-0.038354,-0.044292,-0.268863,-0.302305,-0.728506,...,0.094880,-0.005910,-0.128756,0.000000,-0.025033,0.024939,0.027211,-0.043225,-0.052701,0.238557
3197,2011-02-15,0.181911,1.280784,-0.119291,0.267132,-0.038354,-0.044292,-0.268863,-0.302305,-0.738590,...,0.094880,-0.005910,-0.128756,0.000000,0.000000,0.024939,0.027211,-0.043225,-0.052701,-0.071528
3198,2011-02-16,0.181911,1.280784,-0.119291,0.267132,-0.038354,-0.044292,-0.268863,-0.302305,-0.738590,...,0.094880,-0.005910,-0.128756,-0.072674,0.025033,-0.006048,-0.008212,-0.043225,-0.052701,0.222476
3199,2011-02-17,0.251143,1.280784,-0.078904,0.267132,-0.038354,-0.044292,-0.268863,-0.302305,-0.742823,...,0.094880,-0.005910,-0.128756,0.048449,0.075099,-0.006048,-0.008212,-0.043225,-0.052701,-0.304461
3200,2011-02-18,0.297298,1.280784,-0.051979,0.267132,-0.038354,-0.044292,-0.268863,-0.302305,-0.757638,...,0.094880,-0.005910,-0.128756,0.169572,-0.025033,-0.006048,-0.008212,-0.043225,-0.052701,-0.023678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358,2017-12-26,-1.179653,-0.416750,-1.128964,0.278731,0.177521,-0.348344,0.193708,-1.095999,-0.795236,...,0.004313,0.061065,-0.225522,0.000000,-0.055072,-0.021308,-0.024245,0.164256,-0.052701,0.205112
5359,2017-12-27,-1.202731,-0.416750,-1.155888,0.278731,0.177521,-0.348344,0.193708,-1.095999,-0.778927,...,0.004313,0.061065,-0.225522,0.000000,-0.055072,0.000737,0.000490,0.164256,-0.052701,-0.270325
5360,2017-12-28,-1.202731,-0.416750,-1.155888,0.278731,0.177521,-0.348344,0.193708,-1.095999,-0.808557,...,0.004313,0.061065,-0.225522,0.048449,0.125164,0.000737,0.000490,0.164256,-0.052701,-0.011642
5361,2017-12-29,-1.202731,-0.416750,-1.142426,0.278731,0.177521,-0.348344,0.193708,-1.095999,-0.840801,...,0.004313,0.061065,-0.225522,0.000000,0.000000,0.000737,0.000490,0.164256,-0.052701,0.101619


In [48]:
raw[raw["date"] > "2018-00-00"]

,date,AAA10Y,ASEANTOT,BAA10Y,BUSAPPWNSAUS,BUSAPPWNSAUSYY,CBUSAPPWNSAUS,CBUSAPPWNSAUSYY,CUUR0000SA0R,DEXCHUS,...,PCUOMFGOMFG_DIFF,PCUOMINOMIN_DIFF,SFTPAGRM158SFRBSF_DIFF,T10YIE_DIFF,TEDRATE_DIFF,TLAACBW027NBOG_DIFF,TLBACBW027NBOG_DIFF,TSIFRGHT_DIFF,UNRATE_DIFF,WLEMUINDXD_DIFF
5363,2018-01-02,-1.179653,-0.416750,-1.128964,-3.195322,-1.483809,-3.421834,-1.498519,-1.095999,-0.859849,...,0.004313,0.061065,-0.225522,0.096898,-0.100132,0.000737,0.000490,0.164256,-0.052701,-0.137629
5364,2018-01-03,-1.179653,-0.416750,-1.142426,-3.195322,-1.483809,-3.421834,-1.498519,-1.095999,-0.847399,...,0.004313,0.061065,-0.225522,-0.048449,0.075099,-0.002971,0.005069,0.164256,-0.052701,0.033544
5365,2018-01-04,-1.295040,-0.416750,-1.169350,-3.195322,-1.483809,-3.421834,-1.498519,-1.095999,-0.859227,...,0.004313,0.061065,-0.225522,0.072674,0.000000,-0.002971,0.005069,0.164256,-0.052701,0.570741
5366,2018-01-05,-1.248885,-0.416750,-1.155888,-3.195322,-1.483809,-3.421834,-1.498519,-1.095999,-0.864206,...,0.004313,0.061065,-0.225522,0.000000,0.050066,-0.002971,0.005069,0.164256,-0.052701,-0.351028
5367,2018-01-06,-1.248885,-0.416750,-1.155888,1.553227,0.524462,0.996079,0.723391,-1.095999,-0.864206,...,0.004313,0.061065,-0.225522,0.000000,0.050066,-0.002971,0.005069,0.164256,-0.052701,-0.351028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6332,2021-02-05,-0.302713,-0.579975,-0.590471,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,0.096898,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386
6333,2021-02-08,-0.348868,-0.579975,-0.644321,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,0.024225,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386
6334,2021-02-09,-0.348868,-0.579975,-0.630858,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,0.000000,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386
6335,2021-02-10,-0.325791,-0.579975,-0.617396,0.267132,0.918991,-1.326439,0.449925,-1.372066,-0.890475,...,-0.288952,-0.380179,0.026590,-0.024225,-0.025033,0.010100,0.009509,0.060515,5.428199,-0.442386
